# Executive Compensation vs. Corporate Tax Comparison

Combined analysis comparing executive compensation to corporate tax expense.

**Contents:**
1. Load combined data
2. Compensation-to-tax ratio analysis
3. Outlier identification
4. Sector comparison
5. Visualization (requires matplotlib/plotly)

In [ ]:
# Setup
import sys
from pathlib import Path

src_path = Path.cwd().parent / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

## 1. Load Combined Data

In [ ]:
# Load the combined analysis CSV
output_dir = Path.cwd().parent / "output" / "fortune100"
combined_df = pd.read_csv(output_dir / "compensation_vs_tax.csv")

print(f"Records: {len(combined_df):,}")
print(f"Companies: {combined_df['company'].nunique()}")
print(f"\nColumns: {list(combined_df.columns)}")

In [ ]:
# Preview data
combined_df.head(10)

## 2. Compensation-to-Tax Ratio Analysis

In [ ]:
# Latest year only
latest_year = combined_df['fiscal_year'].max()
latest_df = combined_df[combined_df['fiscal_year'] == latest_year].copy()

# Filter to companies with both valid comp and tax data
valid_df = latest_df[
    (latest_df['total_exec_comp'] > 0) &
    (latest_df['total_tax_expense'] > 0)
].copy()

print(f"Companies with valid data: {len(valid_df)}")

# Calculate ratio (exec comp / tax expense)
# Note: Tax is in millions, comp is in dollars
valid_df['ratio'] = valid_df['total_exec_comp'] / (valid_df['total_tax_expense'] * 1_000_000)

print(f"\nComp-to-Tax Ratio Statistics:")
print(f"  Mean: {valid_df['ratio'].mean():.4f}")
print(f"  Median: {valid_df['ratio'].median():.4f}")
print(f"  (Interpretation: Exec comp is {valid_df['ratio'].median():.2%} of tax expense)")

In [ ]:
# Highest comp-to-tax ratios (exec comp relatively high vs tax)
high_ratio = valid_df.nlargest(10, 'ratio')[['rank', 'company', 'total_exec_comp', 'total_tax_expense', 'ratio']]

print(f"\nHighest Comp-to-Tax Ratios (FY {latest_year}):\n")
print(high_ratio.to_string(index=False))

In [ ]:
# Lowest comp-to-tax ratios (tax relatively high vs exec comp)
low_ratio = valid_df.nsmallest(10, 'ratio')[['rank', 'company', 'total_exec_comp', 'total_tax_expense', 'ratio']]

print(f"\nLowest Comp-to-Tax Ratios (FY {latest_year}):\n")
print(low_ratio.to_string(index=False))

## 3. Outlier Identification

In [ ]:
# Companies with unusually high executive compensation relative to tax
q75 = valid_df['ratio'].quantile(0.75)
q25 = valid_df['ratio'].quantile(0.25)
iqr = q75 - q25
upper_bound = q75 + 1.5 * iqr

outliers = valid_df[valid_df['ratio'] > upper_bound]

print(f"Outliers (ratio > {upper_bound:.4f}): {len(outliers)} companies\n")

if len(outliers) > 0:
    print(outliers[['company', 'total_exec_comp', 'total_tax_expense', 'ratio']].to_string(index=False))

## 4. Sector Comparison

In [ ]:
# Load Fortune 100 registry for sector data
from edgar.data.fortune100 import Fortune100Registry

registry = Fortune100Registry.load_default()
sector_map = {c.ticker: c.sector for c in registry.companies}

# Add sector
valid_df['sector'] = valid_df['ticker'].map(sector_map)

# Sector analysis
sector_analysis = valid_df.groupby('sector').agg({
    'total_exec_comp': 'sum',
    'total_tax_expense': 'sum',
    'ratio': 'mean',
    'company': 'count'
})
sector_analysis.columns = ['Total Comp', 'Total Tax (M)', 'Avg Ratio', 'Companies']
sector_analysis = sector_analysis.sort_values('Avg Ratio', ascending=False)

print(f"\nSector Analysis (FY {latest_year}):\n")
print(sector_analysis.to_string())

## 5. Key Insights

In [ ]:
# Summary statistics
print("=" * 60)
print(f"Fortune 100 Executive Comp vs. Corporate Tax Summary (FY {latest_year})")
print("=" * 60)
print(f"\nCompanies with complete data: {len(valid_df)}")
print(f"\nTotal Executive Compensation: ${valid_df['total_exec_comp'].sum():,.0f}")
print(f"Total Corporate Tax Expense: ${valid_df['total_tax_expense'].sum() * 1_000_000:,.0f}")
print(f"\nOverall Ratio: {valid_df['total_exec_comp'].sum() / (valid_df['total_tax_expense'].sum() * 1_000_000):.4%}")
print(f"\nHighest Paid CEO: {valid_df.loc[valid_df['ceo_comp'].idxmax(), 'ceo_name']}")
print(f"  at {valid_df.loc[valid_df['ceo_comp'].idxmax(), 'company']}")
print(f"  Total: ${valid_df['ceo_comp'].max():,.0f}")
print(f"\nLargest Tax Payer: {valid_df.loc[valid_df['total_tax_expense'].idxmax(), 'company']}")
print(f"  Tax Expense: ${valid_df['total_tax_expense'].max():,.0f}M")

## 6. Optional: Visualization

Uncomment and run if matplotlib is available.

In [ ]:
# # Uncomment to enable visualization
# try:
#     import matplotlib.pyplot as plt
#     
#     fig, ax = plt.subplots(figsize=(12, 8))
#     
#     # Scatter plot: Tax vs Compensation
#     ax.scatter(
#         valid_df['total_tax_expense'],
#         valid_df['total_exec_comp'] / 1_000_000,
#         alpha=0.6
#     )
#     
#     ax.set_xlabel('Corporate Tax Expense ($M)')
#     ax.set_ylabel('Executive Compensation ($M)')
#     ax.set_title(f'Fortune 100: Executive Compensation vs. Corporate Tax (FY {latest_year})')
#     
#     # Add company labels for top 5
#     for idx, row in valid_df.nlargest(5, 'total_exec_comp').iterrows():
#         ax.annotate(
#             row['company'],
#             (row['total_tax_expense'], row['total_exec_comp'] / 1_000_000),
#             fontsize=8
#         )
#     
#     plt.tight_layout()
#     plt.savefig(output_dir / 'comp_vs_tax_scatter.png', dpi=150)
#     plt.show()
#     
# except ImportError:
#     print("matplotlib not available. Install with: pip install matplotlib")

## Next Steps: Streamlit Dashboard

This notebook analysis can be converted to an interactive Streamlit dashboard:

```bash
# Future: Run dashboard
streamlit run src/edgar/dashboard/app.py
```

Features:
- Interactive company selection
- Dynamic filtering by sector, year, rank
- Real-time visualization
- Export functionality